# ICE3333 - Elementos Finitos No-Lineales - Taller 04
## Elasticidad Lineal 3D 

### Resolución mediante elementos finitos en Python

En primer lugar, importamos las librerías necesarias para resolver el problema. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Creamos función que nos permitirá resolver las integrales numericamente en el dominio isoparamétrico. En particular, esta función nos entrega los puntos de integración y pesos asociados. Para lo anterior, recordamos que estas pueden obtenerse a partir de los puntos y pesos en 1D. 

In [2]:
def GaussQuad(k):
    if k==1:
        wl=np.array([2.])
        xil=np.array([0.])        
    elif k==2:
        wl=np.array([1.,1.])
        xil=np.array([-1.,1.])*(1/np.sqrt(3.))
    elif k==3:
        wl=np.array([5/9, 8/9, 5/9])
        xil=np.array([-0.7745966692414834,
                      0,0.7745966692414834])     
    elif k==4:
        wl=np.array([0.347854845, 0.652145155,
                     0.652145155,0.347854845])
        xil=np.array([-0.861136312,-0.339981044,
                      0.339981044,0.861136312])    
    elif k==5:
        wl=np.array([0.236926885 , 0.478628670 ,0.568888889 , 0.478628670 , 0.236926885])
        xil=np.array([-0.906179846,-0.538469310,0,            0.538469310,0.906179846])   
        

    return xil,wl


def HexGaussQuad(k):
    xil,wl=GaussQuad(k)
    xi=np.zeros([k*k*k,3])
    w=np.zeros(k*k*k)
    cont=0
    for i in np.arange(k):
        for j in np.arange(k):
            for l in np.arange(k):
                xi[cont]=np.array([xil[i],xil[j],xil[l]])
                
                w[cont]=wl[i]*wl[j]*wl[l]
                cont=cont+1
    return xi,w


Creamos una función que nos permita obtener las funciones de forma y sus derivadas. Para ello, recordamos que estas pueden ser obtenidas a partir de las expresiones en el dominio isoparamétrico
 \begin{equation}
N_a^e(x)=\hat{N}_a(\hat{\xi}(x))
	\end{equation}


 \begin{equation}
\nabla_x N_a^e(x)=J^{-T} \nabla_\xi \hat{N}_a 
	\end{equation}

In [3]:

def Hex1shp(xeset,xii):
    xi=xii[0]
    eta=xii[1]
    zeta=xii[2]
    
    N1hat=(1/8)*(1-xi)*(1-eta)*(1-zeta)
    N2hat=(1/8)*(1+xi)*(1-eta)*(1-zeta)
    N3hat=(1/8)*(1+xi)*(1+eta)*(1-zeta)
    N4hat=(1/8)*(1-xi)*(1+eta)*(1-zeta)
    N5hat=(1/8)*(1-xi)*(1-eta)*(1+zeta)
    N6hat=(1/8)*(1+xi)*(1-eta)*(1+zeta)
    N7hat=(1/8)*(1+xi)*(1+eta)*(1+zeta)
    N8hat=(1/8)*(1-xi)*(1+eta)*(1+zeta)
     
    Nhat=np.matrix([N1hat,N2hat,N3hat,N4hat,N5hat,N6hat,N7hat,N8hat])   
    
    
    DN11hat=(-1/8)*(eta-1)*(zeta-1)
    DN12hat=(-1/8)*(xi-1)*(zeta-1)
    DN13hat=(-1/8)*(eta-1)*(xi-1)
    
    DN21hat=(1/8)*(eta-1)*(zeta-1)
    DN22hat=(1/8)*(xi+1)*(zeta-1) 
    DN23hat=(1/8)*(eta-1)*(xi+1)    


    DN31hat=(-1/8)*(eta+1)*(zeta-1)    
    DN32hat=(-1/8)*(xi+1)*(zeta-1)  
    DN33hat=(-1/8)*(xi+1)*(eta+1) 
    
    DN41hat=(1/8)*(eta+1)*(zeta-1)  
    DN42hat=(1/8)*(xi-1)*(zeta-1) 
    DN43hat=(1/8)*(xi-1)*(eta+1) 
    
    DN51hat=(1/8)*(eta-1)*(zeta+1)  
    DN52hat=(1/8)*(xi-1)*(zeta+1) 
    DN53hat=(1/8)*(xi-1)*(eta-1) 

    DN61hat=(-1/8)*(eta-1)*(zeta+1)  
    DN62hat=(-1/8)*(xi+1)*(zeta+1) 
    DN63hat=(-1/8)*(xi+1)*(eta-1) 

    DN71hat=(1/8)*(eta+1)*(zeta+1)  
    DN72hat=(1/8)*(xi+1)*(zeta+1) 
    DN73hat=(1/8)*(xi+1)*(eta+1) 

    DN81hat=(-1/8)*(eta+1)*(zeta+1)  
    DN82hat=(-1/8)*(xi-1)*(zeta+1) 
    DN83hat=(-1/8)*(xi-1)*(eta+1) 

            
    DNhat=np.matrix([[DN11hat,DN21hat,DN31hat,DN41hat,DN51hat,DN61hat,DN71hat,DN81hat],
                     [DN12hat,DN22hat,DN32hat,DN42hat,DN52hat,DN62hat,DN72hat,DN82hat],
                     [DN13hat,DN23hat,DN33hat,DN43hat,DN53hat,DN63hat,DN73hat,DN83hat]])
    
    
    
    xhat=(Nhat*xeset.T)
    
    J=(xeset*DNhat.T) 
    detJ=abs(np.linalg.det(J))
    
    N=Nhat
    invJ = np.linalg.inv(J)
    DN=invJ.T*DNhat
    return xhat,N,DN,detJ

Creamos una función que nos permita obtener la matriz de rigidez y vector de fuerzas elemental para elasticidad lineal

In [4]:
def H1kefeElasticity(xeset,C,f,k):
    fe=np.zeros((24,1))
    ke=np.zeros((24,24))
    k=2
    x,w=HexGaussQuad(k)
    for i in np.arange(w.shape[0]):
        xi=np.array([x[i][0],x[i][1],x[i][2]])
        ww=w[i]
        xhat,N,DN,detJ=Hex1shp(xeset,xi)
        for a in np.arange(8):  
                Ba=np.zeros([6,3])
                Ba[0,0]=DN[0,a]
                Ba[1,1]=DN[1,a]
                Ba[2,2]=DN[2,a]
                
                Ba[3,1]=DN[2,a]
                Ba[3,2]=DN[1,a]
                
                Ba[4,0]=DN[2,a]
                Ba[4,2]=DN[0,a]
                
                Ba[5,0]=DN[1,a]
                Ba[5,1]=DN[0,a]
                
                Ba=np.asmatrix(Ba) 
                Na=np.matrix([[N[0,a],0,0  ],
                              [0,N[0,a],0   ],
                              [0,   0, N[0,a]]])
                if a==0:
                    B=Ba
                    N=Na
                else:
                    B=np.hstack((B,Ba)) ## [B1, B2, B3,...B8]  6x24
                    N=np.hstack((N,Na))
        ke=ke+ww*(B.T*C*B)*detJ
        fe=fe+ww*(N.T*f)*detJ   
    
    return ke,fe    

Veremos el ejemplo de 1 elemento. Para ello, creamos los arreglos XYZ e IEN 

In [9]:
L=2
IEN=np.array([[0,1,2,3,4,5,6,7]]) #1x8
XYZ=np.array([[0.  , 0.  , 0.  ],
       [L, 0.  , 0.  ],
       [L , L, 0.  ],
       [0.  , L, 0.  ],
       [0.  , 0.  , L],
       [L , 0.  , L],
       [L , L, L],
       [0.  , L, L]])
    
nnodes=XYZ.shape[0]
nel=IEN.shape[0]   
   
E=80000 
nu=0.15
xeset=XYZ.T
C=(nu*E/((1+nu)*(1-2*nu)))*np.matrix([[1,1,1,0,0,0],
           [1,1,1,0,0,0],
            [1,1,1,0,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,0],
            [0,0,0,0,0,0]])+2*(E/(2*(1+nu)))*np.matrix([[1,0,0,0,0,0],
              [0,1,0,0,0,0],
              [0,0,1,0,0,0],
              [0,0,0,0.5,0,0],
              [0,0,0,0,0.5,0],
              [0,0,0,0,0,0.5]])

        

k=2        
f=np.matrix([0,0,0]).T #peso propio¿

#io.write_points_cells('Malla'+'.vtk',XYZ,{'hexahedron': IEN})
ke,fe=H1kefeElasticity(xeset,C,f,k)


Consideramos el problema de empotrar el elemento en su base, para lo cual construimos los arreglos ID, LM

In [10]:
ID=np.zeros([nnodes,3],dtype=int)
cont=0
for i in np.arange(nnodes):
    if XYZ[i,2]==0 :
        ID[i,0]=-1
        ID[i,1]=-1
        ID[i,2]=-1
    else:
        ID[i,0]=cont
        cont=cont+1
        ID[i,1]=cont
        cont=cont+1
        ID[i,2]=cont
        cont=cont+1

LM=np.zeros([nel,24],dtype=int)
e=0
LM[e,0]=ID[IEN[e,0],0]
LM[e,1]=ID[IEN[e,0],1]
LM[e,2]=ID[IEN[e,0],2]

LM[e,3]=ID[IEN[e,1],0]
LM[e,4]=ID[IEN[e,1],1]
LM[e,5]=ID[IEN[e,1],2]

LM[e,6]=ID[IEN[e,2],0]
LM[e,7]=ID[IEN[e,2],1]
LM[e,8]=ID[IEN[e,2],2]

LM[e,9]=ID[IEN[e,3],0]
LM[e,10]=ID[IEN[e,3],1]
LM[e,11]=ID[IEN[e,3],2]

LM[e,12]=ID[IEN[e,4],0]
LM[e,13]=ID[IEN[e,4],1]
LM[e,14]=ID[IEN[e,4],2]

LM[e,15]=ID[IEN[e,5],0]
LM[e,16]=ID[IEN[e,5],1]
LM[e,17]=ID[IEN[e,5],2]

LM[e,18]=ID[IEN[e,6],0]
LM[e,19]=ID[IEN[e,6],1]
LM[e,20]=ID[IEN[e,6],2]

LM[e,21]=ID[IEN[e,7],0]
LM[e,22]=ID[IEN[e,7],1]
LM[e,23]=ID[IEN[e,7],2]
     

Resolvemos el problema

In [11]:
ndofs=np.max(LM)+1
K=np.zeros((ndofs,ndofs))
F=np.zeros((ndofs,1))
for e in np.arange(nel):
    xeset=np.asmatrix(XYZ[IEN[e]]).T
    ke,fe=H1kefeElasticity(xeset,C,f,k)
    nee=LM.shape[1]
    for p in np.arange(nee):
        P=LM[e,p]
        if P>=0:
            for q in np.arange(nee):
                Q=LM[e,q]
                if Q>=0:
                    K[P,Q]=ke[p,q]
            F[P]=fe[p,0]        


gdl_vert_XL=[]
for i in np.arange(nnodes):
    if XYZ[i,2]==L :
        gdl_vert_XL.append(ID[i,2])
        
PR=10000
FU=L*L*0.25*PR   
F[gdl_vert_XL]=FU
u=np.linalg.solve(K,F)


U0=0 
dispmap=np.zeros(ID.shape)

  
for i in np.arange(ID.shape[0]):
    for j in np.arange(ID.shape[1]):
        if ID[i,j] == -1:
            dispmap[i,j] = U0
        elif ID[i,j] != -1:    
            dispmap[i,j] = u[ID[i,j]]
                

#io.write_points_cells('RES'+'.vtk',XYZ,{'hexahedron': IEN}, point_data={'Desplazamiento':dispmap})

In [12]:
print(dispmap)

[[ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.02542105  0.02542105  0.24573684]
 [-0.02542105  0.02542105  0.24573684]
 [-0.02542105 -0.02542105  0.24573684]
 [ 0.02542105 -0.02542105  0.24573684]]
